<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fbaf575a4be0b9750c41bb10fef20af7a57145a69bf8906172f18007d7668e9a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 10:40:51
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -14.67 K (-0.1%)
Current PnL: -16.90 L (-11.37%)
CY Booked + Current PnL: -5.46 L (-3.67%)
-------------------
Total profit:  2.02 L
Total loss:  -18.92 L
-------------------
Total Booked + Current PnL: 21.52 L (17.68%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.02%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.42 L (57.05%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.15,62.0,H-MC,3.48,195230.0,30560.0,12768.0,-0.44,18.56,6.54,26.31,89.0,2.39,1.43,28.02,X40N,NTT,AC
76,TTKPRESTIG,770.00,102.77,53.0,M-SC,3.70,87384.0,-13393.0,13483.0,-0.97,-13.29,15.43,0.09,245.0,-0.99,0.64,14.80,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213555.0,21813.0,15953.0,-0.18,11.38,7.47,19.69,99.0,1.37,1.56,18.95,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,66.0,H-MC,7.95,112217.0,11713.0,17394.0,0.20,11.65,15.50,28.96,79.0,0.67,0.82,49.58,MH,ATH,METALS
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.95,219227.0,8861.0,19928.0,-0.69,4.21,9.09,13.68,37.0,0.44,1.61,21.43,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-175.00,72.0,M-SC,3.36,86922.0,-15288.0,108679.0,4.08,-14.96,125.03,91.37,208.0,-0.14,0.64,41.10,XY25,NTT,FINANCE
6,ASIANTILES,137.0,7100.00,68.0,L-SC,5.23,78994.0,-14816.0,91436.0,1.84,-15.79,115.75,81.67,269.0,-0.16,0.58,52.98,XR,NTT,CERAMICS
59,SAIL,228.0,47.55,69.0,M-MC,7.54,233297.0,8335.0,159319.0,1.73,3.71,68.29,74.53,192.0,0.05,1.71,37.15,XY24,BTT,STEEL
57,REPCOHOME,880.0,-59.02,56.0,H-SC,6.01,183105.0,-52638.0,241937.0,1.46,-22.33,132.13,80.30,134.0,-0.22,1.34,23.95,XY24,NTT,FINANCE
26,GREENPANEL,537.0,233.33,61.0,M-SC,1.81,148791.0,-34287.0,113260.0,1.41,-18.73,76.12,43.14,230.0,-0.30,1.09,39.96,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-324.58,53.0,H-SC,6.26,124848.0,-38757.0,160068.0,-1.96,-23.69,128.21,74.15,139.0,-0.24,0.91,60.03,XR,NTT,CHEMICALS
53,RAJESHEXPO,518.00,1855.81,61.0,L-SC,2.66,53379.0,-83798.0,83890.0,-1.72,-61.09,157.16,0.07,267.0,-1.00,0.39,32.76,OX40N,NTT,JEWELLERY
55,RELAXO,1176.00,-32.81,53.0,H-SC,1.92,91565.0,-53595.0,129519.0,-1.69,-36.92,141.45,52.31,136.0,-0.41,0.67,22.57,X40N,NTT,FOOTWEAR
36,INDIAMART,4850.92,-52.96,36.0,H-SC,11.49,131493.0,8157.0,125602.0,-1.66,6.61,95.52,108.45,119.0,0.06,0.96,30.45,AR,ATH,MISC
52,QUESS,986.00,-40.50,47.0,M-SC,39.83,58900.0,-6106.0,157033.0,-1.34,-9.39,266.61,232.18,198.0,-0.04,0.43,2.57,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,57.0,H-SC,16.92,201786.0,1832.0,95404.0,0.06,0.92,47.28,48.63,115.0,0.02,1.48,12.57,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.31,47.0,H-MC,2.17,137024.0,1236.0,41518.0,0.22,0.91,30.30,31.48,107.0,0.03,1.00,13.14,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.91,61.0,H-MC,1.93,250052.0,2321.0,72740.0,0.54,0.94,29.09,30.30,92.0,0.03,1.83,14.35,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.94,51.0,H-MC,4.10,199935.0,-885.0,69657.0,-0.18,-0.44,34.84,34.25,80.0,-0.01,1.46,4.73,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,102.77,53.0,M-SC,3.70,87384.0,-13393.0,13483.0,-0.97,-13.29,15.43,0.09,245.0,-0.99,0.64,14.80,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.42,123498.0,-18981.0,38507.0,0.51,-13.32,31.18,13.70,163.0,-0.49,0.90,45.76,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,66.0,H-SC,1.24,228933.0,-40734.0,77059.0,0.44,-15.11,33.66,13.47,138.0,-0.53,1.68,15.79,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,45.0,H-SC,2.10,145475.0,-30428.0,72447.0,-0.35,-17.30,49.80,23.89,149.0,-0.42,1.07,25.64,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-20.01,52.0,H-MC,7.46,108310.0,-23525.0,65755.0,-0.58,-17.84,60.71,32.03,98.0,-0.36,0.79,21.32,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,47.55,69.0,M-MC,7.54,233297.0,8335.0,159319.0,1.73,3.71,68.29,74.53,192.0,0.05,1.71,37.15,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.15,62.0,H-MC,3.48,195230.0,30560.0,12768.0,-0.44,18.56,6.54,26.31,89.0,2.39,1.43,28.02,X40N,NTT,AC
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213555.0,21813.0,15953.0,-0.18,11.38,7.47,19.69,99.0,1.37,1.56,18.95,XY25,NTT,AC
17,CAMS,5211.76,1.71,69.0,H-SC,3.43,115690.0,13686.0,30241.0,1.39,13.42,26.14,43.06,122.0,0.45,0.85,32.83,X40N,ATH,MISC
29,HAVELLS,2069.16,0.91,61.0,H-MC,1.93,250052.0,2321.0,72740.0,0.54,0.94,29.09,30.30,92.0,0.03,1.83,14.35,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,7.95,112217.0,11713.0,17394.0,0.20,11.65,15.50,28.96,79.0,0.67,0.82,49.58,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,0.65,100328.0,8830.0,69919.0,0.17,9.65,69.69,86.07,223.0,0.13,0.73,45.69,XR,NTT,DURABLES
85,WIPRO,420.00,-9.71,59.0,M-LC,5.66,158732.0,7787.0,101668.0,-0.35,5.16,64.05,72.51,53.0,0.08,1.16,11.49,XR,NTT,IT
36,INDIAMART,4850.92,-52.96,36.0,H-SC,11.49,131493.0,8157.0,125602.0,-1.66,6.61,95.52,108.45,119.0,0.06,0.96,30.45,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,66.0,H-MC,6.32,184698.0,4890.0,107513.0,0.91,2.72,58.21,62.52,88.0,0.05,1.35,37.73,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.58,41.0,H-LC,6.60,300211.0,-15631.0,128310.0,-0.63,-4.95,42.74,35.67,5.0,-0.12,2.20,8.22,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-12.80,48.0,H-LC,7.79,220000.0,-31768.0,84986.0,0.86,-12.62,38.63,21.13,27.0,-0.37,1.61,18.34,X40,ATH,PAINTS
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200661.0,523.0,20367.0,-0.29,0.26,10.15,10.44,4.0,0.03,1.47,5.84,X40,NTT,FMCG
30,HCLTECH,1943.91,1.94,51.0,H-LC,9.70,232517.0,-9407.0,72685.0,-0.86,-3.89,31.26,26.15,8.0,-0.13,1.70,9.88,X40,ATH,IT
19,CIPLA,1795.00,-19.78,51.0,H-LC,5.70,213747.0,9247.0,32169.0,-1.14,4.52,15.05,20.25,10.0,0.29,1.57,13.65,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.29,70.0,H-LC,1.48,248468.0,2317.0,76901.0,-0.05,0.94,30.95,32.18,16.0,0.03,1.82,37.32,X200,ATH,IT
41,ITC,452.0,-37.92,50.0,H-LC,2.38,200661.0,523.0,20367.0,-0.29,0.26,10.15,10.44,4.0,0.03,1.47,5.84,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.24,52.0,H-LC,4.18,279466.0,14040.0,42758.0,-1.22,5.29,15.30,21.40,11.0,0.33,2.05,13.01,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,55.0,H-LC,4.95,219227.0,8861.0,19928.0,-0.69,4.21,9.09,13.68,37.0,0.44,1.61,21.43,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.78,51.0,H-LC,5.70,213747.0,9247.0,32169.0,-1.14,4.52,15.05,20.25,10.0,0.29,1.57,13.65,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.75,228915.0,-5229.0,53864.0,0.26,-2.23,23.53,20.77,86.0,-0.10,1.68,4.53,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297219.0,-48737.0,123584.0,-0.47,-14.09,41.58,21.64,1.0,-0.39,2.18,5.29,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200661.0,523.0,20367.0,-0.29,0.26,10.15,10.44,4.0,0.03,1.47,5.84,X40,NTT,FMCG
82,VBL,671.64,-16.58,41.0,H-LC,6.60,300211.0,-15631.0,128310.0,-0.63,-4.95,42.74,35.67,5.0,-0.12,2.20,8.22,X40N,ATH,FMCG
30,HCLTECH,1943.91,1.94,51.0,H-LC,9.70,232517.0,-9407.0,72685.0,-0.86,-3.89,31.26,26.15,8.0,-0.13,1.70,9.88,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.60,65.0,H-LC,26.99,166400.0,-19695.0,232095.0,0.31,-10.58,139.48,114.13,15.0,-0.08,1.22,22.54,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200661.0,523.0,20367.0,-0.29,0.26,10.15,10.44,4.0,0.03,1.47,5.84,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,51.0,H-LC,5.70,213747.0,9247.0,32169.0,-1.14,4.52,15.05,20.25,10.0,0.29,1.57,13.65,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-12.80,48.0,H-LC,7.79,220000.0,-31768.0,84986.0,0.86,-12.62,38.63,21.13,27.0,-0.37,1.61,18.34,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.95,219227.0,8861.0,19928.0,-0.69,4.21,9.09,13.68,37.0,0.44,1.61,21.43,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297219.0,-48737.0,123584.0,-0.47,-14.09,41.58,21.64,1.0,-0.39,2.18,5.29,X40,ATH,IT
39,INFY,2275.00,-16.73,55.0,H-LC,8.19,325102.0,12214.0,159463.0,-0.93,3.90,49.05,54.87,3.0,0.08,2.38,10.81,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200661.0,523.0,20367.0,-0.29,0.26,10.15,10.44,4.0,0.03,1.47,5.84,X40,NTT,FMCG
82,VBL,671.64,-16.58,41.0,H-LC,6.60,300211.0,-15631.0,128310.0,-0.63,-4.95,42.74,35.67,5.0,-0.12,2.20,8.22,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,260976.0,-643.0,119866.0,0.02,-0.25,45.93,45.57,7.0,-0.01,1.91,11.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7100.00,68.0,L-SC,5.23,78994.0,-14816.0,91436.0,1.84,-15.79,115.75,81.67,269.0,-0.16,0.58,52.98,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.22,84480.0,-29069.0,69122.0,0.09,-25.60,81.82,35.27,268.0,-0.42,0.62,106.77,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-175.00,72.0,M-SC,3.36,86922.0,-15288.0,108679.0,4.08,-14.96,125.03,91.37,208.0,-0.14,0.64,41.10,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-3.83,49.0,H-SC,12.51,89548.0,-11415.0,105944.0,0.35,-11.31,118.31,93.63,148.0,-0.11,0.66,33.38,SR,ATH,CHEMICALS
48,MASFIN,398.61,-17.34,50.0,H-SC,12.98,94065.0,-3915.0,25520.0,-0.18,-4.00,27.13,22.05,152.0,-0.15,0.69,36.73,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.0,1104.15,69.0,M-SC,4.57,163791.0,-17352.0,62814.0,0.44,-9.58,38.35,25.10,235.0,-0.28,1.20,31.26,XY24,NTT,HEALTHCARE
60,SAMMAANCAP,326.0,-175.00,72.0,M-SC,3.36,86922.0,-15288.0,108679.0,4.08,-14.96,125.03,91.37,208.0,-0.14,0.64,41.10,XY25,NTT,FINANCE
7,ATULAUTO,844.0,3862.16,62.0,M-SC,5.35,123463.0,-23905.0,75720.0,-0.95,-16.22,61.33,35.16,236.0,-0.32,0.90,26.11,XY24,NTT,AUTO
74,TITAGARH,1548.0,0.96,68.0,H-SC,3.26,165638.0,-26377.0,105263.0,0.46,-13.74,63.55,41.08,158.0,-0.25,1.21,40.36,XY24,NTT,ENGINEERING
80,VAIBHAVGBL,521.0,63.64,68.0,H-SC,6.61,141998.0,-40777.0,151838.0,-1.15,-22.31,106.93,60.77,125.0,-0.27,1.04,27.18,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.07
1,25,43.92
2,50,74.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.55
LC    32.11
MC    23.32
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.35
X40      15.39
X40N     11.96
XR       11.79
AR        8.74
XY25      8.28
OX40N     8.20
X200      1.82
MH        1.70
X5K       1.45
SR        1.30
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.30
H-LC    25.40
H-MC    20.30
M-SC    13.66
M-LC     5.64
M-MC     2.71
L-SC     1.59
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.63,-12.88,74.43
FMCG,11.87,-4.43,40.97
FINANCE,9.79,-15.29,60.44
MISC,7.47,-12.58,75.16
BANKS,6.33,-15.50,76.26
PAINTS,5.89,-14.83,31.95
ELECTRICAL,5.79,-4.15,61.55
AC,3.71,4.62,11.36
INSURANCE,3.44,-6.94,43.62


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2836430.0,22
AR,1302536.0,9
XR,1299309.0,14
X40,745197.0,10
X40N,574432.0,9
OX40N,526379.0,10
XY25,335495.0,6
SR,244175.0,2
X5K,119870.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3411685.0,29
M-SC,1377754.0,17
H-LC,1227125.0,14
H-MC,1094047.0,15
M-LC,380028.0,4
M-MC,307796.0,2
L-SC,244448.0,3
L-MC,60458.0,1
L-LC,38529.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1023014.0      6
M-SC       XY24       796601.0      7
H-SC       AR         789113.0      5
           XR         754637.0      7
H-MC       XY24       490380.0      4
H-LC       X40        461085.0      5
           AR         351961.0      2
M-MC       XY24       307796.0      2
H-SC       X40N       285823.0      4
           OX40N      251171.0      4
           SR         244175.0      2
H-LC       X40N       214343.0      3
H-MC       X40        213138.0      4
M-SC       AR         161462.0      2
L-SC       XR         160558.0      2
M-LC       XY24       158490.0      2
H-MC       XY25       125601.0      2
M-SC       OX40N      125563.0      4
M-LC       X5K        119870.0      1
M-SC       XR         114475.0      2
           XY25       108679.0      1
H-MC       XR         107513.0      1
M-LC       XR         101668.0      1
L-SC       OX40N       83890.0      1
H-LC       X200        76901.0      1
H-MC       X40N        74266.0      2
M-SC       X40         70974.0      1
H-MC       OX40N       65755.0      1
H-SC       MH          63752.0      1
H-LC       XY25        62686.0      2
L-MC       XR          60458.0      1
H-LC       XY24        60149.0      1
L-LC       XY25        38529.0      1
H-MC       MH          17394.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
